We will begin the multinomial log-linear models via a neural networks, then LDA and finally QDA. 

In [6]:
setwd('C:/Users/iceca/Documents/Earthquake_Damage_Predictor')
library(tidyverse)
library(MASS)
library(caret)
library(nnet)
library(randomForest)
library(e1071)

In [2]:
loadFin<- modules::use('Helpers/Load_Final_Data.R')
train <- loadFin$filtered()[[1]]
trainLab <- loadFin$filtered()[[2]]
val <- loadFin$filtered()[[3]]
valLab <- loadFin$filtered()[[4]]
test <- loadFin$filtered()[[5]]

#when R reads the csv, it thinks damage_grade is an integer, so convert it back to a factor
trainLab$damage_grade <- as.factor(trainLab$damage_grade)
valLab$damage_grade <- as.factor(valLab$damage_grade)

manipulate<- modules::use('Helpers/Manipulate.R')
trainFull <- manipulate$combineLab(train, trainLab)
valFull <- manipulate$combineLab(val, valLab)


Joining, by = "building_id"
Joining, by = "building_id"


In [3]:
#saving predictions helper function
savePredictions <- function(model) {
    preds <- cbind(test$building_id, predict(model, subset(test, select=-c(building_id))))
    colnames(preds) <- c("building_id", "damage_grade")
    write.csv(preds, 'Models/Predictions/Random_Forest.csv', row.names=FALSE)
}

In [4]:
trainFull$building_id <- NULL
valFull$building_id <- NULL

In [7]:
model.Multinom <- multinom(as.factor(damage_grade) ~ ., trainFull, maxit=300)

# weights:  189 (124 variable)
initial  value 257668.526186 
iter  10 value 217464.131283
iter  20 value 205705.053699
iter  30 value 205003.169849
iter  40 value 201945.506402
iter  50 value 196033.749889
iter  60 value 192966.323094
iter  70 value 188748.846870
iter  80 value 186924.092821
iter  90 value 185842.187192
iter 100 value 185290.207974
iter 110 value 185071.299722
iter 120 value 184981.660105
iter 130 value 184969.544471
final  value 184969.479414 
converged


In [8]:
prediction <- predict(model.Multinom, valFull)
accuracy <- mean(prediction == valFull$damage_grade)
cat('Accuracy for the neural network was ', accuracy)

Accuracy for the neural network was  0.5875062

### Linear Discriminant Analysis (LDA)

In [10]:
model.LDA <- lda(as.factor(damage_grade)~., data = trainFull)
predictionsVal <- predict(model.LDA, valFull)
accuracy <- mean(predictionsVal$class == valFull$damage_grade)
cat('Accuracy for the LDA was ', accuracy)

Accuracy for the LDA was  0.5797168

### Quadratic Discriminant Analysis (QDA)

In [12]:
model.QDA <- qda(as.factor(damage_grade)~., 
                 data = trainFull)
predictionsVal <- predict(model.QDA, valFull)
accuracy <- mean(predictionsVal$class == valFull$damage_grade)
accuracy

[1] 0.4262308

### Neural Network

In [48]:
#Neuralnet preprocessing

#one hot encode the label
trainLabProcessed <- data.frame(predict(dummyVars(" ~ .", data = trainLab), trainLab) )
valLabProcessed <- data.frame(predict(dummyVars(" ~ .", data = valLab), valLab))

#combine test, val, and train to preprocess together
ntrain <- nrow(train)
nval <- nrow(val)
ntest <- nrow(test)
X <- rbind(train, val, test)
building_id <- X$building_id #extract building_id because it should not be scaled (to assisst in joins)
X <- model.matrix(~.-building_id , X)
X <- scale(X)
X <- data.frame(cbind(X,building_id))
X$X.Intercept. <- NULL
#trainProcessed <- subset( inner_join(X[1:ntrain,], trainLabProcessed) , select=-c(building_id)) 
#valProcessed <- subset(inner_join(X[(1:nval) + ntrain ,], valLabProcessed), select=-c(building_id))
#testProcessed <- subset(X[(1:ntest) + ntrain + nval,], select=-c(building_id))
trainProcessed <- subset(X[1:ntrain,], select=-c(building_id))
valProcessed <- subset(X[(1:nval) + ntrain ,], select=-c(building_id))
testProcessed <- subset(X[(1:ntest) + ntrain + nval,], select=-c(building_id))
trainLab <- subset(trainLab, select=-c(building_id))
valLab <- subset(valLab, select=-c(building_id))

Joining, by = "building_id"
Joining, by = "building_id"


In [7]:
 model.Neuralnet <- ANN2::neuralnetwork(trainProcessed, trainLab, lossFunction = "log",
  rectifierLayers = NA, sigmoidLayers = NA, regression = FALSE,
  standardize = TRUE, learnRate = 5e-03, maxEpochs = 10,
  hiddenLayers = c(5, 5), momentum = 0.9, learnRate = 0.001, verbose = TRUE)

#f = as.formula("damage_grade.1 + damage_grade.2 + damage_grade.3 ~. -building_id")
#nn <- neuralnet(f,
#                data = trainProcessed,
#                hidden = c(5, 5, 5), threshold = 0.001,
#                act.fct = "tanh",
#                linear.output = FALSE,
#                lifesign = "minimal")

ERROR: Error in .doLoadActions(where, attach): error in load action .__A__.1 for package ANN2: Rcpp::loadModule(module = "ANN", what = TRUE, env = ns, loadNow = TRUE): Unable to load module "ANN": cannot allocate vector of size 12.3 Gb


### Random Forests

In [13]:
mtry <- c(5,10,15,20,25)
ntree <- c(80,70,60,50,40) 

createRandomForest <- function(mtry_, ntree_, trainFull, valFull){
    model.RF <- randomForest(as.factor(damage_grade) ~ ., data=trainFull, ntree=ntree_, mtry=mtry_, importance=TRUE)
    #dput(list(model.RF$confusion, modelRF$oob.times, modelRF$, "Models/Random_Forest_Output.txt")
    cat('RESULTS FOR RANDOM FOREST, with mtry=', mtry_, 'ntree=', ntree_, '\n \n')
    
    predictionsVal <- predict(model.RF, valFull)
    accuracy <- mean(predictionsVal == valFull$damage_grade)
    cat('The accuracy for this random forest was',accuracy, '\n')
    
    cat('\n Confusion Matrix \n')
    print(model.RF$confusion)
    
    cat('\n Variables sorted in importance (decreasing)\n')
    imp <- as.data.frame(importance(model.RF))
    print(subset ( imp[order(-imp$MeanDecreaseGini),] ,select=MeanDecreaseGini))
}

In [14]:
createRandomForest(mtry[1],ntree[1], trainFull, valFull)

RESULTS FOR RANDOM FOREST, with mtry= 5 ntree= 80 
 
The accuracy for this random forest was 0.7072254 

 Confusion Matrix 
     1      2     3 class.error
1 9679  12603   285   0.5710994
2 4304 116603 12593   0.1265693
3  379  39132 38962   0.5034980

 Variables sorted in importance (decreasing)
                                       MeanDecreaseGini
geo_level_1_id                             11177.018979
geo_level_2_id                              6397.839110
geo_level_3_id                              5473.540206
area_percentage                             3670.718859
age                                         3605.781455
height_percentage                           2608.603218
foundation_type                             2418.422592
has_superstructure_mud                      1787.318664
other_floor_type                            1567.162650
ground_floor_type                           1555.399178
count_floors_pre_eq                         1424.687495
roof_type                     

In [15]:
createRandomForest(mtry[2],ntree[2], trainFull, valFull)

RESULTS FOR RANDOM FOREST, with mtry= 10 ntree= 70 
 
The accuracy for this random forest was 0.7354284 

 Confusion Matrix 
      1      2     3 class.error
1 11109  11014   444   0.5077325
2  5182 111685 16633   0.1634082
3   544  29953 47976   0.3886305

 Variables sorted in importance (decreasing)
                                       MeanDecreaseGini
geo_level_1_id                             16826.543472
geo_level_3_id                             13922.620851
geo_level_2_id                             13682.425580
age                                         9490.914411
area_percentage                             9469.047563
height_percentage                           5479.170869
foundation_type                             2542.438978
count_families                              2454.679948
ground_floor_type                           2354.130642
has_superstructure_mud                      2259.470949
other_floor_type                            2206.152172
position                 

In [16]:
createRandomForest(mtry[3],ntree[3], trainFull, valFull)

RESULTS FOR RANDOM FOREST, with mtry= 15 ntree= 60 
 
The accuracy for this random forest was 0.7325122 

 Confusion Matrix 
      1      2     3 class.error
1 11522  10521   524   0.4894315
2  5818 109620 18062   0.1788764
3   667  28691 49115   0.3741159

 Variables sorted in importance (decreasing)
                                       MeanDecreaseGini
geo_level_1_id                             17604.238760
geo_level_3_id                             16900.788626
geo_level_2_id                             15348.600766
age                                        12526.516203
area_percentage                            12249.825931
height_percentage                           6527.203651
has_superstructure_mud                      3461.954688
count_families                              2985.995536
foundation_type                             2941.970284
position                                    2543.605053
ground_floor_type                           2366.227052
land_surface_condition   

In [17]:
createRandomForest(mtry[4],ntree[4], trainFull, valFull)

RESULTS FOR RANDOM FOREST, with mtry= 20 ntree= 50 
 
The accuracy for this random forest was 0.7324738 

 Confusion Matrix 
      1      2     3 class.error
1 11704  10252   611   0.4813666
2  6225 108160 19115   0.1898127
3   696  28123 49654   0.3672473

 Variables sorted in importance (decreasing)
                                       MeanDecreaseGini
geo_level_1_id                             17927.502123
geo_level_3_id                             17722.587561
geo_level_2_id                             15470.559789
age                                        13825.498303
area_percentage                            13478.173714
height_percentage                           6823.962831
has_superstructure_mud                      4247.343935
count_families                              3193.416571
position                                    2939.566396
foundation_type                             2770.141749
land_surface_condition                      2368.367962
ground_floor_type        

In [18]:
createRandomForest(mtry[5],ntree[5], trainFull, valFull)

RESULTS FOR RANDOM FOREST, with mtry= 25 ntree= 40 
 
The accuracy for this random forest was 0.7267948 

 Confusion Matrix 
      1      2     3 class.error
1 11722  10198   647   0.4805690
2  6577 107126 19797   0.1975581
3   747  28069 49657   0.3672091

 Variables sorted in importance (decreasing)
                                       MeanDecreaseGini
geo_level_1_id                             18185.827857
geo_level_3_id                             17865.851443
geo_level_2_id                             15268.739331
age                                        14446.945223
area_percentage                            14230.558922
height_percentage                           7184.544210
has_superstructure_mud                      4156.896931
foundation_type                             3516.086029
count_families                              3266.517540
position                                    3027.209899
land_surface_condition                      2345.372841
count_floors_pre_eq      

In [19]:
#best model based on accuracy of the previous models
model.RF <- randomForest(as.factor(damage_grade) ~ ., data=rbind(trainFull,valFull), ntree=ntree[2], mtry=mtry[2], importance=FALSE)


In [20]:
savePredictions(model.RF)

### Support Vector Machines

In [21]:
createSVMModel <- function(kernel, cost, trainFull, valFull, gamma=0.1, degree=1) {
    if (kernel == "linear"){
        cat("Kernel selected as linear")
        model.SVM <- svm(as.factor(damage_grade) ~., data=trainFull, cost=cost, scale=TRUE, kernel="linear")
    }
    if (kernel == "polynomial") {
        cat("Kernel selected as polynomial")
        model.SVM  <- svm(as.factor(damage_grade) ~., data=trainFull, cost=cost, degree=degree, scale=TRUE, kernel="polynomial")
    }
    if (kernel == "radial") {
        cat("Kernel selected as radial")
        model.SVM <- svm(as.factor(damage_grade) ~., data=trainFull, cost=cost,gamma=gamma, scale=TRUE, kernel="radial")
    }
    preds <- predict(model.SVM, valFull)
    acc <- mean(preds == valFull$damage_grade)
    cat('The accuracy for this SVM, with kernel', kernel, 'and cost', cost,'\n\n')
}
gammaSearch <- 10^(-9:3)
costSearch <- 10^(-3:3)
degreeSearch <- 1:5


In [ ]:
library(parallelSVM)
model.SVM <- parallelSVM(as.factor(damage_grade) ~., data=trainFull, sampleSize=0.1,kernel="linear", cost=1)

Warning message:
"package 'parallelSVM' was built under R version 3.6.3"

In [ ]:
preds <- predict(model.SVM, valFull)
acc <- mean(preds == valFull$damage_grade)


In [ ]:
cat('The accuracy for this SVM, with kernel','linear', 'and cost', 0.5,'was',acc,'\n\n')

In [ ]:
#linear search
for (c in c(0.01, 1, 10)){ 
    createSVMModel("linear", c, trainFull, valFull) 
}

In [ ]:
#polynomial
polyParams <- expand.grid(costSearch, degreeSearch)
ntry <- 10
set.seed(5)
params <- sample(1:nrow(polyParams) , ntry, replace=FALSE)
for (i in params){
    c <- polyParams[i,]$cost
    deg <- polyParams[i,]$degree
    cat('PARAMETERS: COST',c,' DEGREE ',deg, '\n')
    createSVMModel("polynomial", c, trainFull, valFull, degree=deg) 
}

In [ ]:
#radial
radParams <- expand.grid(cost=costSearch, gamme=gammaSearch)
ntry <- 10
set.seed(5)
params <- sample(1:nrow(radParams) , ntry, replace=FALSE)
for (i in params){
    c <- radParams[i,]$cost
    gam <- radParams[i,]$gamma
    createSVMModel("polynomial", c, trainFull, valFull, gamma=gam) 
    cat('PARAMETERS: COST',c,' GAMMA ',gam, '\n')
}